# Objective: Finetune I2I LLM w/ LoRA

In [1]:
%load_ext autoreload
%autoreload 2

# device_num = 0

# !export CUDA_VISIBLE_DEVICES=device_num
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]=f"{device_num}"

#utf-8
os.environ['PYTHONIOENCODING'] = 'utf-8'

In [2]:
import torch
print(torch.cuda.device_count())
device = f"cuda:2"
torch.cuda.set_device(2)

from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling, BitsAndBytesConfig
from datasets import load_dataset, Dataset
# from peft import get_peft_model, LoraConfig, TaskType, PeftModel
# from unsloth import FastLanguageModel
import pprint
import json
from pathlib import Path
import transformers
import os


4


In [3]:
max_seq_length = 0     # Unsloth auto supports RoPE Scaling internally!
dtype = None              # None for auto detection
load_in_4bit = False      # Use 4bit quantization to reduce memory usage. Can be False.



if torch.cuda.get_device_capability()[0] >= 8:
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
# attn_implementation = "eager"
print(f"Using {attn_implementation} for attention computation.")
# QLora?

Using flash_attention_2 for attention computation.


In [4]:

current_metadata = json.load(open("metadata.json", "r"))

common_prompt = open("prompt.txt", "r").read()
print(common_prompt)

나는 훌룡한 HVAC 관련 질문 답변을 제공하는 인공지능이다. 사용자의 질문을 받아들여서, 그에 맞는 답변을 제공하는 것이 내 임무이다. 사용자의 질문을 받아들일 때, 다음과 같은 절차를 따라야 한다.
<출력 내용>
1. 'Formalized Input': 사용자의 추상적 질문을 구체화 및 정규화한 결과. 다양한 형태의 질문들을 가장 핵심적이고 근본적인 질문으로 변환한 결과.
2. 'Input Semantic Parsing': Input Semantic Parsing 결과. dict 형태로 구성되며, Temporal, Spatial, Modality를 가짐.
3. 'Strategy': 질문을 해결하기 위한 전략을 답변 전 고민. Objective: 질문의 근본적 의도 및 답변의 목적. Expected Output: 답변의 예상 결과. Step: 답변을 위한 구체으로 쪼개진 단계.
4. 'Instruction Set': 문제 해결을 위해 나의 실제 행동.
Input - Expected Outputs - templates 끼리는 문맥적으로 유사해야함.

<DDL statement>
CREATE TABLE IF NOT EXISTS data_t
(
    id integer NOT NULL DEFAULT nextval('data_t_id_seq'::regclass),
    idu_id integer,
    roomtemp double precision,
    settemp double precision,
    oper boolean,
    "timestamp" timestamp without time zone NOT NULL
)
    
CREATE TABLE IF NOT EXISTS idu_t
(
    id integer NOT NULL DEFAULT nextval('idu_t_id_seq'::regclass),
    name character varying(50) COLLATE pg_catalog."default",
    metadata charac

In [5]:
model_id = 'sh2orc/Llama-3.1-Korean-8B-Instruct'
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'
# model_id = 'Saxo/Linkbricks-Horizon-AI-Korean-Gemma-2-sft-dpo-27B'

model_dir = f"/model/{model_id.replace('/', '-')}"

## Load tokenizer and dataset

In [6]:
# Tokenizer initialization
model = AutoModelForCausalLM.from_pretrained(
    model_id,  
    cache_dir=f"{model_dir}/cache",
    local_files_only=True,
    
    torch_dtype=torch_dtype,
    attn_implementation=attn_implementation,
    # quantization_config=BitsAndBytesConfig(
    #     # load_in_4bit=True,
    #     # bnb_4bit_use_double_quant=True,
    #     # bnb_4bit_quant_type="nf4",
    #     # bnb_4bit_compute_dtype=torch_dtype
    #     load_in_8bit=False if not "27B" in model_id else True,
    #     llm_int8_enable_fp32_cpu_offload=False if not "27B" in model_id else True,
    # ),
    device_map={'': device}
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    cache_dir=f"{model_dir}/cache",
    # local_files_only=True
)


if not os.path.exists(model_dir):
    model.save_pretrained(model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:

tokenizer.padding_side = "left"
print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")

Pad Token id: None and Pad Token: None
EOS Token id: 128009 and EOS Token: <|eot_id|>


In [8]:
def read_dataset(path):
    # the file is originally json-list format
    # we want every first-level elements to be a string itself
    # for example, [{"Hi": "a'b'"}, {"Hi": "c'd'"}] -> ["""{"Hi": "a'b'"}""", """{"Hi": "c'd'"}"""]
    
    with open(path, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
    
    result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in data]
    # print(f"Read {len(result)} examples from {path}")
    # print(f"Type of result: {type(result)}")
    # print(f"Type of result[0]: {type(result[0])}")
    # print(f"Type of result[0]['Input']: {type(result[0]['Input'])}")
    # print(f"Type of result[0]['Response']: {type(result[0]['Response'])}")
    return result

import re

# dataset_tr = Dataset.from_list(read_dataset("training_dataset_v2_directsql_tr.json"))
# dataset_ts = Dataset.from_list(read_dataset("training_dataset_v2_directsql_ts.json"))
dataset_tr = Dataset.from_list(read_dataset("training_dataset_v4_onlyq_tr.json"))
dataset_ts = Dataset.from_list(read_dataset("training_dataset_v4_onlyq_ts.json"))

# dataset = load_dataset("json", data_files="training_dataset_v2_directsql.json")
max_seq_length = 0
def formatting_prompts_func(examples):
    convos = []
    # Iterate through each item in the batch (examples are structured as lists of values)
    for input, response in zip(examples['Input'], examples['Response']):
        global max_seq_length
        response.replace("    ", "")
        answer = {
            "content": f"{response}",
            "role": "assistant"
        }
        if "llama" in model_id.lower():
            prompt = {
                "content": common_prompt,
                "role": "system"
            }
            user_input = {
                "content": f"{current_metadata};{input}",
                "role": "user"
            }
            convos.append([prompt, user_input, answer])
        elif "gemma" in model_id.lower():
            user_input = {
                "content": f"{common_prompt};{current_metadata};{input}",
                "role": "user"
            }
            convos.append([user_input, answer])
        
        
        
        
        # print("Answer length: ", len(response))
        # convos.append([prompt, user_input, answer])
        
        if len(response) + 50 > max_seq_length:
            max_seq_length = len(response) + 50
            print(response)
    
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos]
    # remove \n\nCutting Knowledge Date: BLAH BLAH \nToday Date: BLAH BLAH\n\n using regex
    texts = [re.sub(r'(\nCutting Knowledge Date:.*?\nToday Date:.*?\n\n)', '', text) for text in texts]
    
    return {"text": texts}

dataset_tr = dataset_tr.map(formatting_prompts_func, batched=True)
dataset_ts = dataset_ts.map(formatting_prompts_func, batched=True)

# print(dataset_tr[0]["text"])
print(max_seq_length)
# print(f"seq length: {len(tokenizer.encode(dataset_tr[0]['text']))}")

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

{"Formalized Input": "오늘 우리반과 옆반의 평균 실내온도 차이 알려줘.", "Input Semantic Parsing": {"Temporal": {"오늘": "2022-09-30 00:00:00 ~ 2022-09-30 23:59:59"}, "Spatial": {"우리반": "01_IB5", "옆반": "01_IB7"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "오늘 우리반과 옆반 각각의 평균 실내온도 쿼리 후, 두 값의 차이를 계산함.", "Expected Outputs": ["우리반(19.3℃)이 옆반(17.0℃)보다 2.3℃ 높습니다."], "Step": ["우리반의 오늘 실내온도 쿼리결과 qr_roomtemp_ours 에 저장", "옆반의 오늘 실내온도 쿼리결과 qr_roomtemp_beside 에 저장", "각 반의 실내온도 평균 계산", "각 반의 평균 실내온도와 차이 반환"]}, "Instruction Set": [{"type": "q", "args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp BETWEEN '2022-09-30 00:00:00' AND '2022-09-30 23:59:59'"], "subquery": "idu_id IN (SELECT id FROM idu_t WHERE name IN ('01_IB5'))"}, "result_name": "qr_roomtemp_ours"}, {"type": "q", "args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp BETWEEN '2022-09-30 00:00:00' AND '2022-09-30 23:59:59'"], "subquery": "idu_id IN (SELECT id FROM idu_t WHERE name IN 

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

{"Formalized Input": "우리반 오늘 아침과 저녁의 실내온도 차이 알려줘.", "Input Semantic Parsing": {"Temporal": {"오늘 아침": "2022-09-30 06:00:00 ~ 2022-09-30 09:00:00", "오늘 저녁": "2022-09-30 18:00:00 ~ 2022-09-30 21:00:00"}, "Spatial": {"우리반": "01_IB5"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "우리반 오늘 아침과 저녁의 실내온도 데이터 쿼리 후 평균값을 계산하여 차이를 반환함.", "Expected Outputs": ["오늘 아침(06:00 ~ 09:00, 24.5℃)과 저녁(18:00 ~ 21:00, 27.3℃)의 온도 차이는 2.8℃ 입니다."], "Step": ["2022-09-30 06:00 ~ 09:00 시간대의 roomtemp 데이터 조회해 qr_morning_temp에 저장", "2022-09-30 18:00 ~ 21:00 시간대의 roomtemp 데이터 조회해 qr_evening_temp에 저장", "qr_morning_temp과 qr_evening_temp의 평균값 및 차이 반환"]}, "Instruction Set": [{"type": "q", "args": {"table_name": "data_t", "columns": ["timestamp", "roomtemp"], "conditions": ["timestamp BETWEEN '2022-09-30 06:00:00' AND '2022-09-30 09:00:00'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"}, "result_name": "qr_morning_temp"}, {"type": "q", "args": {"table_name": "data_t", "columns": ["times

## Training config

## Train

In [9]:

torch.cuda.empty_cache()
model = model.to(device)
print(len(dataset_tr))


36


In [10]:
from trl import SFTTrainer, SFTConfig
import numpy as np


per_device_train_batch_size, epochs = 18, 1000 # 8
gradient_accumulation_steps = int(np.ceil(len(dataset_tr) / per_device_train_batch_size))
print(f"Gradient Accumulation Steps: {gradient_accumulation_steps}")

# clear all checkpoints
import shutil
shutil.rmtree(f"{model_dir}/chkpts/full-tuning", ignore_errors=True)

args = SFTConfig(
    # num_train_epochs = 1,
    per_device_train_batch_size = per_device_train_batch_size,  # Controls the batch size per device
    per_device_eval_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,  # Accumulates gradients to simulate a larger batch
    max_steps=gradient_accumulation_steps * epochs,
    # 리소스 제약때문에 batch size를 타협해야하는 경우가 발생 -> micro batch size를 줄이고,
 	# accumulated step을 늘려, 적절한 size로 gradient를 구해 weight update
    # https://www.youtube.com/watch?v=ptlmj9Y9iwE
    warmup_steps = gradient_accumulation_steps,
    learning_rate = 1e-4,             # Sets the learning rate for optimization
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,              # Regularization term for preventing overfitting
    lr_scheduler_type = "cosine",  # Sets the learning rate scheduler
    seed = 3407,                        
    output_dir = f"{model_dir}/chkpts/full-tuning",  # Output directory for checkpoints and predictions     
    report_to = "none",              # Enables Weights & Biases (W&B) logging
    logging_steps = gradient_accumulation_steps,                # Sets frequency of logging to W&B
    logging_strategy = "steps",       # Logs metrics at each specified step
    evaluation_strategy="steps",  # enable evaluation during training
    eval_steps=gradient_accumulation_steps,
    # eval_accumulation_steps=1, # 낮을수록 eval시 사용하는 메모리 줄어듦
    save_steps=gradient_accumulation_steps,
    save_strategy = "steps",               
    load_best_model_at_end = True,    # Loads the best model at the end
    save_only_model = False,           # Saves entire model, not only weights
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    
)


# training_args = SFTConfig(
    
#     output_dir = "/tmp"
# )

trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = dataset_tr,
    eval_dataset = dataset_ts,
    args = args,
)






Gradient Accumulation Steps: 2


/usr/local/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/36 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(


In [11]:
# from unsloth import unsloth_train

# trainer_stats = unsloth_train(trainer)
# print(trainer_stats)

trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacity of 47.41 GiB of which 8.94 MiB is free. Process 1188235 has 42.59 GiB memory in use. Process 1211765 has 4.78 GiB memory in use. Of the allocated memory 4.48 GiB is allocated by PyTorch, and 41.77 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# del base_model
# del peft_model

torch.cuda.empty_cache()

# Specify the checkpoint directory
checkpoint_dir = f"{model_dir}/chkpts/full-tuning/checkpoint-{12}"
# checkpoint_dir = "/model/Bllossom-llama-3.2-Korean-Bllossom-3B/chkpts/r1700_a1500/checkpoint-12"
print(checkpoint_dir)
# Load the tokenizer (ensure it's the same tokenizer used for training)

# Load the base model
peft_model, tokenizer = FastLanguageModel.from_pretrained(
    checkpoint_dir,
    max_seq_length = max_seq_length,
    dtype = dtype,
    quantization_config=BitsAndBytesConfig(
        # load_in_4bit=True,
        # bnb_4bit_use_double_quant=True,
        # bnb_4bit_quant_type="nf4",
        # bnb_4bit_compute_dtype=torch_dtype
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    ),
    device_map=device,
    # cache_dir="/model",
    attn_implementation=attn_implementation,
    cache_dir=f"{model_dir}/cache",
    local_files_only=True
)

tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
# tokenizer.truncation_side = "left"

print(f"Pad Token id: {tokenizer.pad_token_id} and Pad Token: {tokenizer.pad_token}")
print(f"EOS Token id: {tokenizer.eos_token_id} and EOS Token: {tokenizer.eos_token}")
print(f"Padding side: {tokenizer.padding_side}")
# tokenizer.pad_token = tokenizer.eos_token
# 


# Make sure the tokenizer is ready


/model/sh2orc-Llama-3.1-Korean-8B-Instruct/chkpts/r256_a512/checkpoint-12
==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA RTX A6000. Max memory: 47.413 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Pad Token id: 128004 and Pad Token: <|finetune_right_pad_id|>
EOS Token id: 128009 and EOS Token: <|eot_id|>
Padding side: left


In [ ]:
if False:
    # Local saving
    peft_model.save_pretrained("lora_i2i") 
    tokenizer.save_pretrained("lora_i2i")

    # For merging the LoRA adapters with the base model and save the model to 16-bit precision for optimized performance with vLLM, use:
    # # Merge to 16bit
    peft_model.save_pretrained_merged("i2i_merged_16bit", tokenizer, save_method = "merged_16bit",)
    peft_model.save_pretrained_merged("i2i_merged_4bit", tokenizer, save_method = "merged_4bit_forced",)
    # model.push_to_hub_merged("<hf_username/model_name>", tokenizer, save_method = "merged_16bit", token = hf_token)

## Tryout

In [ ]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
if False:
    # del peft_model
    torch.cuda.empty_cache()
    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "i2i_merged_16bit",        # Trained model either locally or from huggingface
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = False,
        attn_implementation=attn_implementation,
        # device_map=[device],
        local_files_only=True
    )
FastLanguageModel.for_inference(peft_model)  # Enable native 2x faster inference


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=256, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=256, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

[Instruction(['q', '이번달 우리반 실내온도가 최고인 날짜 알려줘.', '이번달 우리반 실내온도가 최고인 날짜', '값', Semantic(Temporal=[('이번달', '2022-09-01 00:00:00 ~ 2022-09-30 23:59:59')], Spatial=['우리반'], Modality=['실내온도'], Operation=['최고인'], Target=['날짜'])]), Instruction(['r', "예) '이번달 우리반의 실내 온도가 가장 더웠던 날은 2022년 9월 15일입니다.'"])]

In [ ]:
import re, time

def extract_content(text):
    # Define the regex pattern to extract the content
    pattern = r"<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def run(query):
    start_time = time.time()
    chat = [
        {"role": "system", "content": f"{common_prompt}"},
        # {"role": "metadata", "content": f"{current_metadata}"},
        {"role": "user", "content": f"{current_metadata};{query}"},
    ]
    
    inputs = tokenizer.apply_chat_template(
        chat,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
    
    outputs = peft_model.generate(
        input_ids = inputs,
        max_new_tokens = max_seq_length,
        use_cache = True,
        pad_token_id = tokenizer.eos_token_id,
        eos_token_id = tokenizer.eos_token_id,
    )
    
    response = tokenizer.batch_decode(outputs)[0]
    # print(response)
    # print("Extracting..")
    parsed_response = extract_content(response)
    pprint.pprint(f"Query: {query}, Time: {time.time() - start_time}")
    print(parsed_response)
    
    parsed_response_dict = eval(parsed_response)
    
    return parsed_response, parsed_response_dict
    
    # text_streamer = transformers.TextStreamer(
    #     tokenizer, 
    # skip_prompt = True
    # )
    # _ = peft_model.generate(
    #     input_ids = inputs, 
    #     streamer = text_streamer, 
    #     max_new_tokens = max_seq_length, 
    #     use_cache = True,
    #     pad_token_id = tokenizer.eos_token_id,
    #     eos_token_id = tokenizer.eos_token_id,
    # )
    

In [ ]:
result = run("오늘 우리반과 옆반의 평균 온도차이 알려줘")
# parsed_response_dict = eval(result)

# print(parsed_response_dict)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


'Query: 오늘 우리반과 옆반의 평균 온도차이 알려줘, Time: 76.17268538475037'
None


TypeError: eval() arg 1 must be a string, bytes or code object

In [ ]:

run("오늘 우리반 평균온도 알려줘")

run("어제 옆반 온도 평균 알려줘")
run("현재 우리반 실내온도 알려줘")

'Query: 오늘 우리반 평균온도 알려줘, Time: 11.774609327316284'
{"Formalized Input": "오늘 우리반(01_IB5) 평균 실내온도를 알려줘.", "Input Semantic Parsing": {"Temporal": {"오늘": "2022-09-30 00:00:00 ~ 2022-09-30 23:59:59"}, "Spatial": {"우리반": "01_IB5"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "오늘 우리반의 실내온도 데이터를 쿼리 후 평균값 계산", "Expected Outputs": ["오늘 우리반의 평균 실내온도는 25.5℃입니다."], "Step": ["오늘 우리반의 실내온도 데이터를 쿼리결과 qr_roomtemp_today에 저장", "qr_roomtemp_today에서 실내온도의 평균값 계산", "실내온도의 평균값 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp BETWEEN '2022-09-30 00:00:00' AND '2022-09-30 23:59:59'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB5')"}}, "result": {"name": "qr_roomtemp_today"}}, {"type": "o", "python_script_to_process": "avg_roomtemp = qr_roomtemp_today['roomtemp'].mean()", "return": ["avg_roomtemp"]}, {"type": "r", "templates": ["오늘 우리반의 평균 실내온도는 {avg_roomtemp}℃입니다."]}]}
'Query: 어제 옆반 온도 평균 알려

('{"Formalized Input": "현재 01_IB5 실내온도 알려줘.", "Input Semantic Parsing": {"Temporal": {"현재": "2022-09-30 12:00:00"}, "Spatial": {"01_IB5": "우리반"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "01_IB5의 실내온도 데이터를 현재 시점으로 쿼리 후 반환", "Expected Outputs": ["01_IB5의 실내온도는 24℃입니다."], "Step": ["01_IB5의 실내온도 데이터를 현재 시점으로 쿼리결과 qr_roomtemp_01_IB5에 저장", "qr_roomtemp_01_IB5에서 실내온도 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp = \'2022-09-30 12:00:00\'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = \'01_IB5\')"}}, "result": {"name": "qr_roomtemp_01_IB5"}}, {"type": "r", "python_script_to_process": "roomtemp = qr_roomtemp_01_IB5[\'roomtemp\'].values[0]", "return": ["roomtemp"]}, {"type": "o", "templates": ["01_IB5의 실내온도는 {roomtemp}℃입니다."]}]}',
 {'Formalized Input': '현재 01_IB5 실내온도 알려줘.',
  'Input Semantic Parsing': {'Temporal': {'현재': '2022-09-30 12:00:00'},
   'Spatial': {'01_IB5': '우리반'}

In [ ]:
run("롯데캐슬의 현재 온도 알려줘")
run("10년전 우리반 온도 알려줘")

'Query: 롯데캐슬의 현재 온도 알려줘, Time: 5.028217315673828'
{"Formalized Input": "롯데캐슬의 현재 실내온도를 알려줘.", "Input Semantic Parsing": {"Temporal": {"현재": "2022-09-30 12:00:00"}, "Spatial": {"롯데캐슬": "unknown"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "unknown spatial entity를 user에게 물어보고, 그에 맞는 답변 제공", "Expected Outputs": ["unknown spatial entity를 다시 한번 확인해주세요."], "Step": ["unknown spatial entity를 user에게 물어보기"]}, "Instruction Set": [{"type": "r", "templates": ["unknown spatial entity를 다시 한번 확인해주세요."]}]}
'Query: 10년전 우리반 온도 알려줘, Time: 15.704979181289673'
{"Formalized Input": "10년전 우리반 실내온도를 알려줘.", "Input Semantic Parsing": {"Temporal": {"10년전": "2012-09-30 00:00:00 ~ 2012-09-30 23:59:59"}, "Spatial": {"우리반": "01_IB5"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "10년전 우리반 실내온도의 데이터를 쿼리 후 반환", "Expected Outputs": ["10년전 우리반의 실내온도 데이터는 다음과 같습니다. 2012-09-30 00:00:00 ~ 2012-09-30 23:59:59 사이에 평균 24.5℃, 최저 20℃, 최고 30℃였습니다."], "Step": ["10년전 우리반의 실내온도 데이터를 쿼리해 qr_roomtem

('{"Formalized Input": "10년전 우리반 실내온도를 알려줘.", "Input Semantic Parsing": {"Temporal": {"10년전": "2012-09-30 00:00:00 ~ 2012-09-30 23:59:59"}, "Spatial": {"우리반": "01_IB5"}, "Modality": {"실내온도": "roomtemp"}}, "Strategy": {"Objective": "10년전 우리반 실내온도의 데이터를 쿼리 후 반환", "Expected Outputs": ["10년전 우리반의 실내온도 데이터는 다음과 같습니다. 2012-09-30 00:00:00 ~ 2012-09-30 23:59:59 사이에 평균 24.5℃, 최저 20℃, 최고 30℃였습니다."], "Step": ["10년전 우리반의 실내온도 데이터를 쿼리해 qr_roomtemp_10years_ago에 저장", "qr_roomtemp_10years_ago에서 평균, 최저, 최고 온도 계산", "계산 결과 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["roomtemp"], "conditions": ["timestamp BETWEEN \'2012-09-30 00:00:00\' AND \'2012-09-30 23:59:59\'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = \'01_IB5\')"}}, "result": {"name": "qr_roomtemp_10years_ago"}}, {"type": "o", "python_script_to_process": "avg_temp = qr_roomtemp_10years_ago[\'roomtemp\'].mean(); min_temp = qr_roomtemp_10years_ago[\'roomtemp\'].min(); max_temp = qr_roo

In [ ]:
run("지금 옆반 에어컨 상태 알려줘")

'Query: 지금 옆반 에어컨 상태 알려줘, Time: 9.772225141525269'
{"Formalized Input": "지금 옆반 에어컨 전원이 켜져있어?", "Input Semantic Parsing": {"Temporal": {"지금": "2022-09-30 12:00:00"}, "Spatial": {"옆반": "01_IB7"}, "Modality": {"에어컨 전원": "oper"}}, "Strategy": {"Objective": "지금 옆반 에어컨 전원 상태를 쿼리 후 반환", "Expected Outputs": ["옆반 에어컨 전원이 켜져있습니다."], "Step": ["지금 옆반 에어컨 전원 상태를 쿼리해 qr_aircon_oper에 저장", "qr_aircon_oper 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["oper"], "conditions": ["timestamp = '2022-09-30 12:00:00'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = '01_IB7')"}}, "result": {"name": "qr_aircon_oper"}}, {"type": "o", "python_script_to_process": "oper = qr_aircon_oper['oper'].values[0]", "return": ["oper"]}, {"type": "r", "templates": ["옆반 에어컨 전원이 {oper}입니다."]}]}


('{"Formalized Input": "지금 옆반 에어컨 전원이 켜져있어?", "Input Semantic Parsing": {"Temporal": {"지금": "2022-09-30 12:00:00"}, "Spatial": {"옆반": "01_IB7"}, "Modality": {"에어컨 전원": "oper"}}, "Strategy": {"Objective": "지금 옆반 에어컨 전원 상태를 쿼리 후 반환", "Expected Outputs": ["옆반 에어컨 전원이 켜져있습니다."], "Step": ["지금 옆반 에어컨 전원 상태를 쿼리해 qr_aircon_oper에 저장", "qr_aircon_oper 반환"]}, "Instruction Set": [{"type": "q", "run": {"args": {"table_name": "data_t", "columns": ["oper"], "conditions": ["timestamp = \'2022-09-30 12:00:00\'"], "subquery": "idu_id = (SELECT id FROM idu_t WHERE name = \'01_IB7\')"}}, "result": {"name": "qr_aircon_oper"}}, {"type": "o", "python_script_to_process": "oper = qr_aircon_oper[\'oper\'].values[0]", "return": ["oper"]}, {"type": "r", "templates": ["옆반 에어컨 전원이 {oper}입니다."]}]}',
 {'Formalized Input': '지금 옆반 에어컨 전원이 켜져있어?',
  'Input Semantic Parsing': {'Temporal': {'지금': '2022-09-30 12:00:00'},
   'Spatial': {'옆반': '01_IB7'},
   'Modality': {'에어컨 전원': 'oper'}},
  'Strategy': {'Objective': '지금 옆반 